In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [4]:
# Set the path to the dataset
dataset_path = "/kaggle/input/covid19-xray-dataset-train-test-sets/xray_dataset_covid19"

In [5]:
# Set the path to the train and test directories
train_dir = os.path.join(dataset_path, "/kaggle/input/covid19-xray-dataset-train-test-sets/xray_dataset_covid19/train")
test_dir = os.path.join(dataset_path, "/kaggle/input/covid19-xray-dataset-train-test-sets/xray_dataset_covid19/test")

In [6]:
# Set the image dimensions
img_width, img_height = 224, 224

In [7]:
# Set the batch size
batch_size = 32

In [8]:
# Create the train and test data generators
train_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

Found 148 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [9]:
# Build the model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [10]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [11]:

# Train the model
epochs = 10

model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // batch_size,
    epochs=epochs
)

Epoch 1/10
4/4 [==============================] - 14s 2s/step - loss: 0.6875 - accuracy: 0.5345
Epoch 2/10
4/4 [==============================] - 11s 3s/step - loss: 0.6419 - accuracy: 0.6293
Epoch 3/10
4/4 [==============================] - 11s 2s/step - loss: 0.4982 - accuracy: 0.7500
Epoch 4/10
4/4 [==============================] - 11s 2s/step - loss: 0.3637 - accuracy: 0.8621
Epoch 5/10
4/4 [==============================] - 11s 3s/step - loss: 0.2444 - accuracy: 0.9224
Epoch 6/10
4/4 [==============================] - 11s 2s/step - loss: 0.1669 - accuracy: 0.9310
Epoch 7/10
4/4 [==============================] - 11s 2s/step - loss: 0.1183 - accuracy: 0.9397
Epoch 8/10
4/4 [==============================] - 12s 3s/step - loss: 0.1599 - accuracy: 0.9297
Epoch 9/10
4/4 [==============================] - 12s 3s/step - loss: 0.1737 - accuracy: 0.9219
Epoch 10/10
4/4 [==============================] - 12s 3s/step - loss: 0.1459 - accuracy: 0.9453


In [13]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(
    test_generator,
    steps=test_generator.n // batch_size
)

1/1 [==============================] - 2s 2s/step - loss: 0.0234 - accuracy: 1.0000


In [14]:
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

Test Loss: 0.023359952494502068
Test Accuracy: 1.0


In [16]:
from tensorflow.keras.preprocessing import image

In [19]:
# Set the path to the input image from the test data
input_image_path = "/kaggle/input/covid19-xray-dataset-train-test-sets/xray_dataset_covid19/test/PNEUMONIA/SARS-10.1148rg.242035193-g04mr34g0-Fig8a-day0.jpeg"

# Load and preprocess the input image
img = image.load_img(input_image_path, target_size=(img_width, img_height))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = img_array / 255.0  # Normalize the pixel values

In [20]:
# Make predictions on the preprocessed input image
predictions = model.predict(preprocessed_img)
predicted_label = "Pneumonia" if predictions[0] > 0.5 else "Normal"
print("Predicted Label:", predicted_label)

1/1 [==============================] - 0s 50ms/step
Predicted Label: Pneumonia
